In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras import Model, layers

#import torch
#import torch.nn as nn

batch_size = 128
conv1_filters = 32
conv2_filters = 64
fc1_units = 1024

(x_train, y_train), (x_test, y_test) = mnist.load_data() # load MNIST data
x_train,x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
x_train,x_test = x_train / 255, x_test / 255

train_data = tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

class CNN(Model):
  def __init__(self):
    super(CNN,self).__init__()  #입력 데이터 크기 28*28*1
    self.conv1 = layers.Conv2D(32,kernel_size=3,activation=tf.nn.relu) # 채널 크기 ((28 - 3) / 1(stride)) + 1 = 26*26*32
    self.maxpool1 = layers.MaxPool2D(2, strides=2) # 채널 크기 ((26 - 2) / 2(stride)) + 1 = 13*13*32
    self.conv2 = layers.Conv2D(64,kernel_size=3,activation=tf.nn.relu ) # 채널 크기 ((13 - 3) / 1(stride)) + 1 = 11*11*64
    self.maxpool2 = layers.MaxPool2D(2, strides=2) # 채널 크기 ((11 - 2) / 2(stride)) + 1 = 5.5(소수점은 버림) = 5*5*64
    self.flatten = layers.Flatten()
    self.fc1 = layers.Dense(1024) # 노드 개수 = 1024
    self.out = layers.Dense(10) # 출력 노드 = 10

  def call(self,x):
    x = tf.reshape(x,[-1, 28, 28, 1])
    x = self.conv1(x)
    x = self.maxpool1(x)
    x = self.conv2(x)
    x = self.maxpool2(x)
    x = self.flatten(x)
    x = self.fc1(x)
    x = self.out(x)

    return x


CNN_model = CNN()

optimizer = tf.optimizers.Adam(0.01)

@tf.function
def cross_entropy_loss(x, y):
  y = tf.cast(y, tf.int64)
  loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits=x)
  return tf.reduce_mean(loss)

@tf.function
def accuracy(x, y):
  correct = tf.equal(tf.argmax(x,1), tf.cast(y, tf.int64))
  return tf.reduce_mean(tf.cast(correct, tf.float32), axis = -1)

@tf.function
def train_step(x, y):
  with tf.GradientTape() as g:
    y_pred = CNN_model(x)
    loss = cross_entropy_loss(y_pred, y)
    #loss = nn.CrossEntropyLoss(y_pred, y)
    trainable_variables = CNN_model.trainable_variables
  gradients = g.gradient(loss, trainable_variables)
  optimizer.apply_gradients(zip(gradients, trainable_variables))

avg_loss = 0
tot_batch = int(x_train.shape[0]/batch_size)

for epoch in range(10):
  for iter, (batch_x, batch_y) in enumerate(train_data.take(300), 1):
    train_step(batch_x, batch_y)
    current_loss = cross_entropy_loss(CNN_model(batch_x),batch_y)
    #current_loss = nn.CrossEntropyLoss(CNN_model(batch_x),batch_y)

  avg_loss = avg_loss + current_loss/tot_batch
  if iter % 10 == 0:
    print("\nEpoch : ", epoch)
    print("Iteration : ", iter)
    print("Loss : ", avg_loss.numpy())
    print("Accuracy of test = ", accuracy(CNN_model(x_test), y_test).numpy())
    print("------------------------------------------------------------------")
    #print("Epoch: %d, Iteration: %d, Loss = %f, Accuracy of test: %f"% (epoch, iter, avg_loss, accuracy(CNN_model(x_test), y_test)))

11490434/11490434 [==============================] - 1s 0us/step

Epoch :  0
Iteration :  300
Loss :  0.00027381294
Accuracy of test =  0.9698
------------------------------------------------------------------

Epoch :  1
Iteration :  300
Loss :  0.0003316561
Accuracy of test =  0.9712
------------------------------------------------------------------

Epoch :  2
Iteration :  300
Loss :  0.0006268171
Accuracy of test =  0.9625
------------------------------------------------------------------

Epoch :  3
Iteration :  300
Loss :  0.00077686075
Accuracy of test =  0.972
------------------------------------------------------------------

Epoch :  4
Iteration :  300
Loss :  0.0008934253
Accuracy of test =  0.9611
------------------------------------------------------------------

Epoch :  5
Iteration :  300
Loss :  0.0011007012
Accuracy of test =  0.942
------------------------------------------------------------------

Epoch :  6
Iteration :  300
Loss :  0.0012112373
Accuracy of test =  0

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras import Model, layers

#import torch
#import torch.nn as nn

batch_size = 128
conv1_filters = 32
conv2_filters = 64
fc1_units = 1024

(x_train, y_train), (x_test, y_test) = mnist.load_data() # load MNIST data
x_train,x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
x_train,x_test = x_train / 255, x_test / 255

train_data = tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

class CNN(Model):
  def __init__(self):
    super(CNN,self).__init__()  #입력 데이터 크기 28*28*1
    self.conv1 = layers.Conv2D(32,kernel_size=3,activation=tf.nn.relu) # 채널 크기 ((28 - 3) / 1(stride)) + 1 = 26*26*32
    self.maxpool1 = layers.MaxPool2D(2, strides=2) # 채널 크기 ((26 - 2) / 2(stride)) + 1 = 13*13*32
    self.conv2 = layers.Conv2D(64,kernel_size=3,activation=tf.nn.relu ) # 채널 크기 ((13 - 3) / 1(stride)) + 1 = 11*11*64
    self.maxpool2 = layers.MaxPool2D(2, strides=2) # 채널 크기 ((11 - 2) / 2(stride)) + 1 = 5.5(소수점은 버림) = 5*5*64
    self.flatten = layers.Flatten()
    self.fc1 = layers.Dense(1024) # 노드 개수 = 1024
    self.out = layers.Dense(10) # 출력 노드 = 10

  def call(self,x):
    x = tf.reshape(x,[-1, 28, 28, 1])
    x = self.conv1(x)
    x = self.maxpool1(x)
    x = self.conv2(x)
    x = self.maxpool2(x)
    x = self.flatten(x)
    x = self.fc1(x)
    x = self.out(x)

    return x


CNN_model = CNN()

optimizer = tf.optimizers.Adam(0.01)

@tf.function
def cross_entropy_loss(x, y):
  y = tf.cast(y, tf.int64)
  loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits=x)
  return tf.reduce_mean(loss)

@tf.function
def accuracy(x, y):
  correct = tf.equal(tf.argmax(x,1), tf.cast(y, tf.int64))
  return tf.reduce_mean(tf.cast(correct, tf.float32), axis = -1)

@tf.function
def train_step(x, y):
  with tf.GradientTape() as g:
    y_pred = CNN_model(x)
    loss = cross_entropy_loss(y_pred, y)
    #loss = nn.CrossEntropyLoss(y_pred, y)
    trainable_variables = CNN_model.trainable_variables
  gradients = g.gradient(loss, trainable_variables)
  optimizer.apply_gradients(zip(gradients, trainable_variables))


avg_loss = 0
tot_batch = int(x_train.shape[0]/batch_size)
epoch_loss = []
acc_loss = 0

for epoch in range(10):
  avg_loss = 0
  tot_batch = int(x_train.shape[0]/batch_size)
  for iter, (batch_x, batch_y) in enumerate(train_data.take(100), 1):
    train_step(batch_x, batch_y)
    current_loss = cross_entropy_loss(CNN_model(batch_x),batch_y)
    #current_loss = nn.CrossEntropyLoss(CNN_model(batch_x),batch_y)
    #acc_loss += current_loss
  #epoch_loss.append(acc_loss / iter)

  avg_loss = avg_loss + current_loss/tot_batch
  if iter % 10 == 0:
    print("\nEpoch : ", epoch)
    print("Iteration : ", iter)
    print("Loss : ", avg_loss.numpy())
    #print("epoch_Loss : ", epoch_loss)
    print("Accuracy of test = ", accuracy(CNN_model(x_test), y_test).numpy())
    print("------------------------------------------------------------------")
    #print("Epoch: %d, Iteration: %d, Loss = %f, Accuracy of test: %f"% (epoch, iter, avg_loss, accuracy(CNN_model(x_test), y_test)))


Epoch :  0
Iteration :  100
Loss :  0.00012820243
Accuracy of test =  0.963
------------------------------------------------------------------

Epoch :  1
Iteration :  100
Loss :  9.349916e-05
Accuracy of test =  0.9689
------------------------------------------------------------------

Epoch :  2
Iteration :  100
Loss :  4.3221182e-05
Accuracy of test =  0.9686
------------------------------------------------------------------

Epoch :  3
Iteration :  100
Loss :  0.00019004833
Accuracy of test =  0.9686
------------------------------------------------------------------

Epoch :  4
Iteration :  100
Loss :  0.0001548598
Accuracy of test =  0.9654
------------------------------------------------------------------

Epoch :  5
Iteration :  100
Loss :  6.0744e-05
Accuracy of test =  0.9652
------------------------------------------------------------------

Epoch :  6
Iteration :  100
Loss :  5.5950986e-05
Accuracy of test =  0.9698
----------------------------------------------------------